In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Including all modules

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image
import keras
import matplotlib.pyplot as plt

In [ ]:
#Defining directory for training and testing

train_dir = '/content/drive/My Drive/mask detection/Mask Dataset/Train'
validation_dir = '/content/drive/My Drive/mask detection/Mask Dataset/Validation'

In [ ]:
#making data ready to be served to the mode, using datagen

train_datagen = ImageDataGenerator(rescale = 1./255.)
test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 10,
                                                    class_mode = 'binary', 
                                                    target_size = (50, 50))     

validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 10,
                                                          class_mode  = 'binary', 
                                                          target_size = (50, 50))

In [ ]:
#Making Model Sequencial

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(50, 50, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')   #We can use softmax also using 2 dense
])

#Compiling model
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.98):
      print("\nReached 98.0% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
callback = myCallback()

In [ ]:
# Model Summary
model.summary()
keras.utils.plot_model(model, "Mask Detector.png")

In [ ]:
#Trainng model

history = model.fit(
      train_generator,
      steps_per_epoch=75, 
      epochs=50,
      callbacks = [callback],
      validation_data=validation_generator,
      validation_steps=35
      )

In [ ]:
#Save the model to use it later and to save time next time
model.save('maskDetector.h5') 

In [ ]:
# Getting Accuracy of model

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'y', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'y', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
status = ['Mask','Without Mask']

path = 'custom img/without.jpg'

img = image.load_img(path, target_size=(50, 50))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

output = model.predict(x)
output = output[0][0]


tag = int(output)

print(status[tag])